# Test plane extraction for a single point cloud

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import time
import open3d as o3d
import plotly.graph_objects as go

import planeslam.general as general
from planeslam.mesh import LidarMesh
from planeslam.scan import Scan
from planeslam.clustering import cluster_mesh_graph_search, plot_clusters
from planeslam.extraction import scan_from_clusters
from planeslam.scan import pc_to_scan
import planeslam.io as io

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

Read Point Cloud from pcd file

In [ ]:
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PCs = io.read_lidar_bin(binpath)
P = PCs[0]

# Convert points to ENU
P = general.NED_to_ENU(P)

Run everything

In [ ]:
scan = pc_to_scan(P)

In [ ]:
# Plot the scan
fig = go.Figure(data=[general.pc_plot_trace(P)]+scan.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

Run step by step

In [ ]:
# Downsample the points
P = general.downsample(P, factor=2, axis=0)

In [ ]:
# Plot the points 
fig = go.Figure(data=general.pc_plot_trace(P))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()

Cluster the points

In [ ]:
# Create the mesh
start_time = time.time()
mesh = LidarMesh(P)
print("elapsed time: ", time.time() - start_time)

In [ ]:
start_time = time.time()
mesh.prune(edge_len_lim=10)
print("elapsed time: ", time.time() - start_time)

In [ ]:
# Cluster the mesh with graph search
start_time = time.time()
clusters, avg_normals = cluster_mesh_graph_search(mesh)
print("elapsed time: ", time.time() - start_time)

In [ ]:
#%lprun -f cluster_mesh_graph_search cluster_mesh_graph_search(mesh)

In [ ]:
# Plot mesh
fig = go.Figure(data=mesh.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Plot clusters
plot_clusters(P, mesh, clusters)

Extract planes

In [ ]:
from planeslam.extraction import planes_from_clusters
# planes, vertices, faces = scan_from_clusters(mesh, clusters, avg_normals)
# scan = Scan(planes, vertices, faces)
start_time = time.time()
planes, basis = planes_from_clusters(mesh, clusters, avg_normals)
scan = Scan(planes, basis)
print("elasped time: ", time.time() - start_time)

In [ ]:
# Plot the scan
fig = go.Figure(data=scan.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()

In [ ]:
# Find extraction basis based on normals
basis = np.zeros((3,3))
basis[:,2] = avg_normals[0]  # choose first cluster's normal as z
dps = np.asarray(avg_normals) @ basis[:,2]
orth_idxs = np.nonzero(np.abs(dps) < 0.2)[0]  # indices of normals approximately orthonormal to z
basis[:,0] = avg_normals[orth_idxs[0]]  # choose the first one as x
basis[:,1] = np.cross(basis[:,2], basis[:,0])

In [ ]:
from planeslam.clustering import mesh_cluster_pts

i = 9
n = avg_normals[i][:,None]
c = clusters[i]
cluster_pts = mesh_cluster_pts(mesh, c)  # Extract points from cluster

In [ ]:
from planeslam.extraction import bd_plane_from_pts_basis

plane_pts = bd_plane_from_pts_basis(cluster_pts, n, basis)

In [ ]:
n.flatten() @ np.linalg.inv(basis).T

In [ ]:
np.linalg.inv(basis) @ n

In [ ]:
# Plot the points 
fig = go.Figure(data=general.pc_plot_trace(cluster_pts))
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
plane_z = np.median(cluster_pts[:,1])
cluster_pts = cluster_pts[np.abs(cluster_pts[:,1] - plane_z) < 0.1]

In [ ]:
cluster_pts

In [ ]:
plane_pts = np.empty((4,3))

# Project to basis
pts_proj = cluster_pts @ np.linalg.inv(basis).T

In [ ]:
np.median(pts_proj[:,2])

In [ ]:
# Plot the points 
fig = go.Figure(data=general.pc_plot_trace(pts_proj))
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
pts_reproj = pts_proj @ basis.T

In [ ]:
# Plot the points 
fig = go.Figure(data=general.pc_plot_trace(pts_reproj))
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Use normal to determine which dimensions to extract bounding box from
plane_idx = np.argsort(np.linalg.norm(np.hstack((basis, -basis)) - n, axis=0))[0] % 3
#plane_idx = np.argsort(np.linalg.norm(basis - n, axis=0))[0] % 3
axes = {0,1,2}  # x,y,z
axes.remove(plane_idx)
axes = list(axes)

# Find 2D bounding box of points within plane
# Orders points counterclockwise with respect to the normal (i.e. right hand rule)
plane_pts[:,plane_idx] = pts_proj[0,plane_idx]
min = np.amin(pts_proj[:,axes], axis=0)
max = np.amax(pts_proj[:,axes], axis=0)

for i, ax in enumerate(axes):
    if i == 0:  # First coordinate
        if n[plane_idx] > 0:  # Positively oriented normal
            if plane_idx == 0 or plane_idx == 2:  # x or z normal
                plane_pts[:,ax] = np.array([min[i], max[i], max[i], min[i]])  # Case 1
            else: # y normal
                plane_pts[:,ax] = np.array([max[i], min[i], min[i], max[i]])  # Case 2
        else:  # Negatively oriented normal
            if plane_idx == 0 or plane_idx == 2:  # x or z normal
                plane_pts[:,ax] = np.array([max[i], min[i], min[i], max[i]])  # Case 2
            else: # y normal
                plane_pts[:,ax] = np.array([min[i], max[i], max[i], min[i]])  # Case 1
    else:  # Second coordinate
        plane_pts[:,ax] = np.array([min[i], min[i], max[i], max[i]])  # Case 3

# Project back to standard basis
plane_pts = plane_pts @ basis.T

In [ ]:
pts_proj[:,axes]

In [ ]:
# Plot the points 
fig = go.Figure(data=general.pc_plot_trace(plane_pts))
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
plane_pts